In [1]:
#the final
import sys
sys.path.append("..")

In [2]:
import os
from models.grapheme_to_phoneme import GraphemeToPhoneme
import tensorflow as tf
import numpy as np
from utils.dataset_utils import build_dataset
from models.grapheme_to_phoneme import GraphemeToPhoneme
import tensorflow as tf
import pickle
from tqdm import *
from __future__ import division

from compiler.ast import flatten

/root/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The compiler package is deprecated and removed in Python 3.x.
  if sys.path[0] == '':


# Define parametes

In [ ]:
train_parameters = {
    "lr": 0.001,
    "decay_steps": 1000,
    "decay_rate": 0.85,
    "dropout_prob": 0.95
}

model_parameters = {
  #embedding_size": 50,
  "num_units": 30,#随便
  "num_layers": 3,
  "num_beams": 5
}


end_token = 75

num_steps = 1 #5

save_energy = 2

# Train model

In [ ]:
%store -r words_final2
%store -r words_len_final2
#metadata_file = "../../deepvoice2/cmu/cmu.pkl"
metadata_file = "../cmu_data/cmu.pkl"
train_file = "../cmu_data/cmu_data.npz"
train_file2 = "/home/deepvoice/deepvoice2/cmu/VCTK_data2.npz"

with open(metadata_file, "r") as read_file:
    meta = pickle.load(read_file)
char2id = meta["char2id"]
id2char = meta["id2char"]
phoneme2id = meta["phoneme2id"]
id2phoneme = meta["id2phoneme"]

input_vocab_size = len(char2id)#35
output_vocab_size= len(phoneme2id)#76

end_token = phoneme2id["<eos>"]

data = np.load(train_file)
data2=np.load(train_file2)

with tf.Session() as sess:
    characters = tf.placeholder(tf.int32, [None, None])
    characters_seq_len = tf.placeholder(tf.int32, [None])
    phonemes = tf.placeholder(tf.int32, [None, None])
    phonemes_seq_len = tf.placeholder(tf.int32, [None])
    
    prediction_characters = tf.placeholder(tf.int32, [None, None])
    prediction_characters_seq_len = tf.placeholder(tf.int32, [None])
    
    model = GraphemeToPhoneme(input_vocab_size, output_vocab_size, end_token, model_parameters)
    
    #train_op_tf, loss_tf, global_step_tf, summary_tf, decoder_output_tf ,prediction_tf= model.build_train_operations(
    train_op_tf, loss_tf, global_step_tf, summary_tf, decoder_output_tf = model.build_train_operations(
        characters, characters_seq_len, phonemes, phonemes_seq_len, train_parameters
    )
    
    prediction_tf,prediction_seq_len_tf = model.build_prediction(prediction_characters, prediction_characters_seq_len, True)
    


    train_writer = tf.summary.FileWriter('../log/train_grapheme_to_phoneme_model_notebook/train', sess.graph)
    
    tf.global_variables_initializer().run()
   
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    saver = tf.train.Saver(max_to_keep=3, keep_checkpoint_every_n_hours=3)
    train=0
    if train==0:
        tf.add_to_collection('prediction',prediction_tf)
        for _ in tqdm(xrange(num_steps)):
            out = sess.run([
                train_op_tf,
                global_step_tf,
                loss_tf,
                summary_tf,
                decoder_output_tf,
            #prediction_tf
            ], feed_dict={
                characters: data["X"],
                characters_seq_len: data["X_seq_len"],
                phonemes: data["Y"],
                phonemes_seq_len: data["Y_seq_len"]
            })
            _, global_step, loss, summary, decoder_output= out
        #,prediction
            train_file2 = "../../deepvoice2/cmu/VCTK_data.npz"
            data2 = np.load(train_file2)
        #prediction=sess.run([prediction_tf],feed_dict={prediction_characters:data2["X"],
        #                                               prediction_characters_seq_len:data2["X_seq_len"]})

            train_writer.add_summary(summary, global_step)

        # detect gradient explosion
            if loss > 1e8 and global_step > 500:
                print('loss exploded')
                break

        #if global_step % save_energy == 0 and global_step != 0:

            print('saving weights')
        #print np.shape(prediction),prediction
            print(loss)
            #在测试集上算模型精确度？
            #correct_prediction = tf.equal(tf.argmax(decoder_output_tf, 1), tf.argmax(phonemes, 1))
            #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            #print "Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]})
            #print "Accuracy:", accuracy.eval({characters:data["X"],characters_seq_len:...,phonemes: data["Y"],...:...})
            if not os.path.exists('../weights/train_grapheme_to_phoneme_model_notebook/'):
                os.makedirs('../weights/train_grapheme_to_phoneme_model_notebook/')
            saver.save(sess, '../weights/train_grapheme_to_phoneme_model_notebook/model.ckpt', global_step=global_step)
             
        
          
        coord.request_stop()
        coord.join(threads)
    else:
        
        saver.restore(sess, '../weights/train_grapheme_to_phoneme_model_notebook/model.ckpt-1')
        graph = tf.get_default_graph()
        for i in tqdm(range(1)):
            prediction,prediction_seq_len=sess.run([prediction_tf,prediction_seq_len_tf],
                                                   feed_dict={prediction_characters:words_final2,
                                                           prediction_characters_seq_len:words_len_final2})
        print prediction,np.shape(prediction),prediction_seq_len
        #print data2["X"],np.shape(data2["X"]),data2["X_seq_len"],np.shape(data2["X_seq_len"])
        %store prediction
            
        

        coord.request_stop()
        coord.join(threads)
   

total_arg_size 57
total_arg_size 87
shape_w (87, 60)
total_arg_size 57
total_arg_size 87
shape_w (87, 30)
total_arg_size 57
total_arg_size 87
shape_w (87, 60)
total_arg_size 57
total_arg_size 87
shape_w (87, 30)
total_arg_size 60
total_arg_size 90
shape_w (90, 60)
total_arg_size 60
total_arg_size 90
shape_w (90, 30)
total_arg_size 60
total_arg_size 90
shape_w (90, 60)
total_arg_size 60
total_arg_size 90
shape_w (90, 30)
total_arg_size 60
total_arg_size 90
shape_w (90, 60)
total_arg_size 60
total_arg_size 90
shape_w (90, 30)
total_arg_size 60
total_arg_size 90
shape_w (90, 60)
total_arg_size 60
total_arg_size 90
shape_w (90, 30)
total_arg_size 71
total_arg_size 101
shape_w (101, 60)
total_arg_size 71
total_arg_size 101
shape_w (101, 30)
total_arg_size 30
total_arg_size 60
shape_w (60, 60)
total_arg_size 30
total_arg_size 60
shape_w (60, 30)
total_arg_size 30
total_arg_size 60
shape_w (60, 60)
total_arg_size 30
total_arg_size 60
shape_w (60, 30)
total_arg_size 30
shape_w (30, 71)
total_a

  0%|          | 0/1 [00:00<?, ?it/s]